In [1]:
import pandas as pd
import numpy as np

In [2]:
from pathlib import Path

In [3]:
dPath = Path("../docs/dumps")

In [4]:
import pickle

In [5]:
with open(dPath / "train_data.pkl", 'rb') as filename:
    train_data = pickle.load(filename)

In [6]:
with open(dPath / "valid_data.pkl", 'rb') as filename:
    valid_data = pickle.load(filename)

In [7]:
X_train = train_data.drop("Detected", axis=1)

In [8]:
y_train = train_data.Detected

In [9]:
X_valid = valid_data.drop("Detected", axis=1)

In [10]:
y_valid = valid_data.Detected

In [11]:
with open(dPath / "rf_exp_04_names.pkl", 'rb') as filename:
    names = pickle.load(filename)

In [12]:
X_train = X_train[names] 

In [13]:
X_valid = X_valid[names]

In [14]:
X_train.head()

,numExecuted,MutatorClass,numAssertInTC,numTestCover,numAssertInTM,ppavcc,cchalsteadCumulativeBugs,cchalsteadCumulativeVolume,ppRVF,ppnumberOfMethods,...,cchalsteadEffort,ccmaintainabilityIndexNC,mmhalsteadVocabulary,pptcc,mmexternalMethodsCalled,ppinstability,ppmaxcc,mmhalsteadEffort,mmvariablesReferenced,ccunweightedClassSize
0,14,2,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
1,14,6,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
2,14,6,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
3,14,0,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
4,14,0,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0


In [15]:
from imblearn.over_sampling import ADASYN

In [16]:
sm = ADASYN(random_state=42, n_jobs=-1,  n_neighbors=5)

In [17]:
%time X_train, y_train = sm.fit_resample(X_train, y_train)

Wall time: 56.3 s


In [18]:
import lightgbm

In [19]:
lgbm = lightgbm.LGBMClassifier(
        seed=42,
        n_estimators=3000,
        objective='binary',
        n_jobs=-1)

In [20]:
%time lgbm.fit(X_train,y_train)

Wall time: 5min


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=3000, n_jobs=-1, num_leaves=31, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, seed=42,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [21]:
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

In [22]:
def conf_matr(m):
    y_pred = m.predict(X_valid)
    print(classification_report(y_valid, y_pred))

In [23]:
def print_score(m):
    res = [m.score(X_train, y_train), m.score(X_valid, y_valid), roc_auc_score(y_valid, m.predict(X_valid))]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [24]:
print_score(lgbm)

[0.8462676887772919, 0.6996414602346805, 0.598811442700733]


In [25]:
conf_matr(lgbm)

              precision    recall  f1-score   support

       False       0.39      0.40      0.40     16657
        True       0.80      0.80      0.80     50839

    accuracy                           0.70     67496
   macro avg       0.60      0.60      0.60     67496
weighted avg       0.70      0.70      0.70     67496



In [27]:
with open(dPath / "lgbm_exp_01.pkl", 'wb') as filename:
    pickle.dump(lgbm,filename)